<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_4_3_BERT_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. LoRA - BERT 모델을 사용한 PEFT
BERT 모델을 사용하여 PEFT(LoRA) 사용

In [ ]:
!pip install "transformers" "bitsandbytes" "trl" "datasets" "evaluate" "matplotlib-venn" "evaluate"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00


## 허깅페이스 계정에 로그인

In [ ]:
# hugging face login
from huggingface_hub import notebook_login
notebook_login()

## BERT 모델 로드

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# BERT 로드
base_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=6
)

base_model

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 데이터세트 로드

In [ ]:
from datasets import load_dataset

# 데이터세트
dataset = load_dataset("dair-ai/emotion")
dataset['train'][0]

README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': 'i didnt feel humiliated', 'label': 0}

## 토크나이저 설정

In [ ]:
# 토크나이저
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized = dataset.map(preprocess, batched=True)
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_ds = tokenized["train"]
eval_ds  = tokenized["validation"]

train_ds[0]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'labels': tensor(0),
 'input_ids': tensor([  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,  

##  PEFT - Lora 설정과 학습

In [ ]:
from peft import LoraConfig, get_peft_model
import evaluate

# LoRA 구성
lora_config = LoraConfig(
    task_type="SEQ_CLS",  # 시퀀스 분류
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["query", "key", "value", "output.dense"],
)

# LoRA 적용
model = get_peft_model(base_model, lora_config)

# Trainer 구성
training_args = TrainingArguments(
    output_dir="./lora_bert_mrpc",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="steps",
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to="none",
)

# 평가 메트릭
metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return metric.compute(predictions=preds, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 8. 학습 시작
trainer.train()

/tmp/ipython-input-1542683702.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
20,1.800500,1.757964,0.284500
40,1.737400,1.704329,0.351500
60,1.703100,1.664235,0.352500
80,1.659000,1.637356,0.352500
100,1.659100,1.616975,0.352500
120,1.629400,1.603476,0.352500
140,1.619500,1.594790,0.352000
160,1.611600,1.588681,0.352000
180,1.597700,1.583135,0.352000
200,1.570300,1.577623,0.352000


TrainOutput(global_step=3000, training_loss=1.1338222920099894, metrics={'train_runtime': 1522.4465, 'train_samples_per_second': 31.528, 'train_steps_per_second': 1.971, 'total_flos': 3192948965376000.0, 'train_loss': 1.1338222920099894, 'epoch': 3.0})

## 성능 평가 (Evaluate)

In [ ]:
eval_results = trainer.evaluate()
print(f"최종 정확도(Accuracy): {eval_results['eval_accuracy']:.4f}")

최종 정확도(Accuracy): 0.6775


## 모델 저장 (Save)
LoRA는 모델 전체가 아니라 학습된 '어댑터(Adapter)' 부분만 저장하면 됩니다. 용량이 매우 작아서 효율적입니다.

In [ ]:
# 학습된 LoRA 모델(어댑터) 저장
save_dir = "./bert-emotion-lora"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"모델이 '{save_dir}' 폴더에 저장되었습니다.")

모델이 './bert-emotion-lora' 폴더에 저장되었습니다.


## 실제 테스트 (Inference)
저장된 모델이 새로운 문장을 보고 감정을 잘 맞추는지 직접 테스트해 봅니다.

In [ ]:
# 테스트: 문장을 넣어 감정 예측해보기
import torch

# 테스트할 문장
text = "I feel so happy and excited today!"

# 입력 변환
inputs = tokenizer(text, return_tensors="pt").to(model.device)

# 예측
with torch.no_grad():
    logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()

# 결과 출력 (Emotion 데이터셋 라벨 매핑)
# 0: sadness, 1: joy, 2: love, 3: anger, 4: fear, 5: surprise
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
print(f"입력 문장: {text}")
print(f"예측 감정: {labels[predicted_class_id]}")

입력 문장: I feel so happy and excited today!
예측 감정: joy


# 예측

In [ ]:
def textEmotionLoRA(text):
    # 입력 변환
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    # 예측
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()

    # 결과 출력 (Emotion 데이터셋 라벨 매핑)
    # 0: sadness, 1: joy, 2: love, 3: anger, 4: fear, 5: surprise
    labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
    print(f"입력 문장: {text}")
    print(f"예측 감정: {labels[predicted_class_id]}")

textEmotionLoRA('Fuck')

In [ ]:
import torch.nn.functional as F
from collections import Counter

def analyze_emotion_probabilities(text):
    # Tokenize and predict
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        logits = model(**inputs).logits

    # Convert logits to probabilities
    probs = F.softmax(logits, dim=-1)[0]

    labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

    print(f"Input: '{text}'")
    print("Confidence Scores:")
    for label, prob in zip(labels, probs):
        print(f"  {label}: {prob:.4f}")
    print("-" * 30)

# 1. Analyze the specific sentences
analyze_emotion_probabilities('What on earth')
analyze_emotion_probabilities('I am so angry')

# 2. Check Training Data Balance
print("\nDataset Label Distribution:")
labels_list = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
train_labels = dataset['train']['label']
label_counts = Counter(train_labels)
total = len(train_labels)

for i, label in enumerate(labels_list):
    count = label_counts[i]
    percentage = (count / total) * 100
    print(f"{label}: {count} ({percentage:.1f}%)")

모델이 두 문장을 모두 **'슬픔(sadness)'**으로 예측한 결정적인 이유는 학습 데이터의 불균형 때문입니다.

데이터 불균형 (Data Imbalance):

**슬픔(29.2%)**과 **기쁨(33.5%)**이 데이터의 60% 이상을 차지합니다.
반면 **놀람(3.6%)**과 **분노(13.5%)**는 데이터가 매우 적습니다.
모델은 잘 모를 때, 가장 많이 공부한 정답(슬픔/기쁨)을 선택하려는 경향(편향)이 있습니다.
문장별 상세 이유:

'What on earth': 보통 '놀람(Surprise)' 표현이지만, 놀람 데이터가 고작 3.6%뿐이라 모델이 학습을 제대로 못 했습니다. 그래서 애매한 부정적 감정인 '슬픔'으로 찍은 것입니다.
'I am so angry': 모델도 **'분노(Anger)'**일 확률을 **32%**로 꽤 높게 봤습니다. 하지만 '슬픔' 데이터가 '분노'보다 2배 이상 많다 보니, **"화난 것 같지만, 내가 더 잘 아는 슬픔일 확률이 높아(49%)"**라고 판단해버린 것입니다.
이 문제를 해결하려면 부족한 감정(놀람, 분노)의 데이터를 더 모아서 학습시켜야 합니다.